<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/keltner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
!pip install pandas pandas-ta matplotlib yfinance
!pip install --upgrade pandas-datareader


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf
from datetime import date as date
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt


In [99]:
file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)

In [100]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')

unwanted = {'INDO-RE.NS','CDSL.NS','HERIT-RE.NS','ZODIAC.NS','EMAMIPAP.NS','DIGJAMLMTD.NS','GUJRAFFIA.NS','NIRAJISPAT.NS','KALYANI.NS','ARTEMISMED.NS'}
ath_list = [e for e in ath_list if e not in unwanted]

#print (ath_list)

def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ticker))
    return results

results = paralel_func(ath_list)

In [101]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

#pd.set_option('display.max_rows', None)

ticker_list = ticker_list.query('Last_price > ATH * 0.9 and Last_price > two_EMA and Last_price < 500')
ticker_list.reset_index(drop=True, inplace=True)

In [102]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today() + relativedelta(days=1)

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df = yf.download(tickers, start= startdate, end= enddate)

[*********************100%***********************]  81 of 81 completed


In [103]:
pd.set_option('display.max_rows', 10)
#df

In [104]:
from time import strftime
import datetime
prev_day = (date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
to_day = date.today().strftime('%Y-%m-%d')

In [105]:
ax_df = df.copy()
df1 = pd.DataFrame()

for ticker in tickers:
  test = ta.kc(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=20, scalar=1.2, mamode='ema', tr=True)
  test.columns = pd.MultiIndex.from_product([test.columns, [ticker]])
  df1 = pd.concat([df1, test], axis=1)
  df1.sort_index()
  
ax_df = pd.concat([ax_df, df1], axis=1)
ax_df.reset_index()
ax_df = ax_df.drop(['Adj Close','Volume'], level=0, axis=1)

In [106]:
ax_df= ax_df.swaplevel(0, 1, 1).sort_index(axis=1)
ax_df = ax_df.tail(7)


In [107]:
ax_df = ax_df.T

In [108]:
for ticker in tickers:
  for column in ax_df.columns:
    if ax_df.loc[(ticker,'Close'), column] < ax_df.loc[(ticker,'KCLe_20_1.2'), column]:
      print(ticker , column)

MANOMAY.NS 2023-02-03 00:00:00
PETRONET.NS 2023-02-03 00:00:00
RECLTD.NS 2023-02-13 00:00:00
